## BLAST results

The sequence matches 100% identically to the complete genome of Candidatus Carsonella ruddii HC isolate Thao2000. It has an e-value of 0. It also matches other organsisms. Though the e-value for all matches are 0 the score for the first match is much greater. 

## Circos Plot

In [5]:
from pycirclize import Circos
from pycirclize.parser import Gff

# Load in files
fasta_file = "refs/Cutibacterium_acnes.fna"
gff_file = "results/Cutibacterium_acnes/Cutibacterium_acnes/Cutibacterium_acnes.gff"

gff = Gff(gff_file)

# Initialize Circos instance with genome size
seqid2size = gff.get_seqid2size()
# If there's only one sequence, extract its size
if len(seqid2size) == 1:
    genome_size = list(seqid2size.values())[0]
    sectors = {"NZ_AP019723.1": genome_size}
space = 0 if len(sectors) == 1 else 2
circos = Circos(sectors, space=space)
circos.text(f"Cutibacterium acnes\n\nGCF_006739385.1", size=14)

# Extract CDS features
seqid2features = gff.get_seqid2features(feature_type="CDS")

# Plot CDS features
for sector in circos.sectors:
    # Setup track for features plot
    f_cds_track = sector.add_track((95, 100))
    f_cds_track.axis(fc="lightgrey", ec="none", alpha=0.5)
    r_cds_track = sector.add_track((90, 95))
    r_cds_track.axis(fc="lightgrey", ec="none", alpha=0.5)
    # Plot forward/reverse strand CDS
    features = seqid2features[sector.name]
    for feature in features:
        if feature.location.strand == 1:
            f_cds_track.genomic_features(feature, plotstyle="arrow", fc="salmon", lw=0.5)
        else:
            r_cds_track.genomic_features(feature, plotstyle="arrow", fc="skyblue", lw=0.5)

    # Plot 'gene' qualifier label if exists
    labels, label_pos_list = [], []
    for feature in features:
        start = int(feature.location.start)
        end = int(feature.location.end)
        label_pos = (start + end) / 2
        gene_name = feature.qualifiers.get("gene", [None])[0]
        if gene_name is not None:
            labels.append(gene_name)
            label_pos_list.append(label_pos)
    f_cds_track.xticks(label_pos_list, labels, label_size=6, label_orientation="vertical")

    # Plot xticks (interval = 10 Kb)
    r_cds_track.xticks_by_interval(
        10000, outer=False, label_formatter=lambda v: f"{v/1000:.1f} Kb"
    )

circos.savefig("C_acnes.png")

{'NZ_AP019723.1': 2494738}
